# 最简单的 MarkdownReader 示例

结论：

- MarkdownReader 会根据 `md` 文件的标题，拆分出多个 `Document`
- 可以通过继承类，覆盖方法，实现在 `Document` 对象增加 metadata 数据的功能 -- 文件路径

In [1]:
%%time

from pathlib import Path
from llama_index.readers.file import MarkdownReader

filepath="./data/test.md"
content = MarkdownReader().load_data(Path(filepath))

CPU times: user 1.55 s, sys: 260 ms, total: 1.81 s
Wall time: 1.42 s


In [3]:
len(content)

5

In [4]:
content

[Document(id_='6f2d6fe4-bfc1-498d-a181-f5b00603fcc5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\n主要步骤\n\n- 注册微信小程序\n- 安装 “微信开发者工具”\n- 在 “微信开发者工具” 中创建原型项目\n- 进行原生小程序开发\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='56dcc8de-b3c4-4b04-a9a8-7c572586de7b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\n注册微信小程序\n\n地址： https://mp.weixin.qq.com/\n\n需要创建一个单独的邮箱，用于注册:\n\n- https://qiye.aliyun.com/ 使用 postmaster 账号登录\n- 创建新的邮件组，小程序01/mini_01，按照顺序排列\n- 通过邮件组成员账号接收激活小程序\n\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b658864e-3981-4a71-be6e-c5d284fd5abc', embedding=Non

In [23]:
from typing import Any, Dict, List, Optional, Tuple
from fsspec import AbstractFileSystem
from llama_index.core.schema import Document

class MyReader(MarkdownReader):

    def load_data(
        self,
        file: Path,
        extra_info: Optional[Dict] = None,
        fs: Optional[AbstractFileSystem] = None,
    ) -> List[Document]:
        """Parse file into string."""
        tups = self.parse_tups(file, fs=fs)
        results = []
        # TODO: don't include headers right now
        for header, value in tups:
            print(header,file)
            if header is None:
                results.append(Document(text=value, metadata=extra_info or {}))
            else:
                results.append(
                    Document(text=f"\n\n{header}\n{value}", metadata=extra_info or {})
                )
            results[-1].metadata['file']=file.as_posix()

        return results

In [24]:
content2 = MyReader().load_data(Path(filepath))
len(content2)

主要步骤 data/test.md
注册微信小程序 data/test.md
安装“微信开发者工具” data/test.md
在 “微信开发者工具” 中创建原型项目 data/test.md
进行原生小程序开发 data/test.md


5

In [25]:
content2

[Document(id_='e7b44df4-b738-4466-8f79-6d255078829f', embedding=None, metadata={'file': 'data/test.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\n主要步骤\n\n- 注册微信小程序\n- 安装 “微信开发者工具”\n- 在 “微信开发者工具” 中创建原型项目\n- 进行原生小程序开发\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='fba7cb12-48a7-4c85-aa92-67f024adcd82', embedding=None, metadata={'file': 'data/test.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n\n注册微信小程序\n\n地址： https://mp.weixin.qq.com/\n\n需要创建一个单独的邮箱，用于注册:\n\n- https://qiye.aliyun.com/ 使用 postmaster 账号登录\n- 创建新的邮件组，小程序01/mini_01，按照顺序排列\n- 通过邮件组成员账号接收激活小程序\n\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='754f25f7